<a href="https://colab.research.google.com/github/yejerlasubbu/AA79/blob/master/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/productreviews.csv')

In [ ]:
df.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [ ]:
df=df.drop(['asin','name','date','verified','helpfulVotes'],axis=1)
df.head()

,rating,title,body
0,3,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,5,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,3,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


In [ ]:
df['sentiment']=df['rating'].apply(lambda x:'negative' if x<=3 else 'positive')
reviews_df=df[['rating','sentiment','title','body']]
reviews_df.head()

,rating,sentiment,title,body
0,3,negative,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,1,negative,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,5,positive,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,3,negative,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,4,positive,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


In [ ]:
reviews_df.head(n=10)

,rating,sentiment,title,body
0,3,negative,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,1,negative,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,5,positive,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,3,negative,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,4,positive,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...
5,4,positive,Worked great for me,"Hello, I have this phone and used it until I d..."
6,5,positive,Wanna cool Nokia? You have it here!,Cool. Cheap. Color: 3 words that describe the ...
7,4,positive,Problem with 3588i universal headset,"The 3599i is overall a nice phone, except that..."
8,5,positive,cool phone!!!!!!!!,"I've never owned a Nokia phone before, so this..."
9,3,negative,Pissed off-a little bit,ok well im in school and i need the text messa...


DATA PREPROCESSING

In [ ]:
import nltk
from nltk.corpus import wordnet
def get_wordnet_pos(pos_tag):
  if pos_tag.startswith('J'):
    return wordnet.ADJ
  elif pos_tag.startswith('V'):
    return wordnet.VERB
  elif pos_tag.startswith('N'):
    return wordnet.NOUN
  elif pos_tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [ ]:
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
  text=str(text).lower()
  text=[word.strip(string.punctuation) for word in text.split(" ")]
  text=[word for word in text if not any(c.isdigit() for c in word)]
  stop=stopwords.words('english')
  text=[x for x in text if x not in stop]
  text=[t for t in text if len(t)>0]
  pos_tags=pos_tag(text)
  text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
  text = [t for t in text if len(t) > 1]
  text = " ".join(text)
  return(text)

#clean the data
reviews_df['review_clean']=reviews_df['body']
reviews_df["review_clean"] = reviews_df["review_clean"].apply(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
reviews_df['review_clean'].head(10)

0    samsung awhile absolute doo doo read review de...
1    due software issue nokia sprint phone's text m...
2    great reliable phone also purchase phone samsu...
3    love phone really need one expect price bill r...
4    phone great every purpose offer except day buy...
5    hello phone use decide buy flip phone problem ...
6    cool cheap color word describe nokia perfectly...
7    overall nice phone except nokia make universal...
8    i've never own nokia phone first really like p...
9    ok well im school need text message ive phone ...
Name: review_clean, dtype: object

In [ ]:
reviews_df.head()

,rating,sentiment,title,body,review_clean
0,3,negative,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,samsung awhile absolute doo doo read review de...
1,1,negative,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,due software issue nokia sprint phone's text m...
2,5,positive,Love This Phone,"This is a great, reliable phone. I also purcha...",great reliable phone also purchase phone samsu...
3,3,negative,"Love the Phone, BUT...!","I love the phone and all, because I really did...",love phone really need one expect price bill r...
4,4,positive,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,phone great every purpose offer except day buy...


In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
comment_list=list()
lines=reviews_df['review_clean'].values.tolist()

for line in lines:
  tokens=word_tokenize(line)
  comment_list.append(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
reviews_df.head()

,rating,sentiment,title,body,review_clean
0,3,negative,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,samsung awhile absolute doo doo read review de...
1,1,negative,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,due software issue nokia sprint phone's text m...
2,5,positive,Love This Phone,"This is a great, reliable phone. I also purcha...",great reliable phone also purchase phone samsu...
3,3,negative,"Love the Phone, BUT...!","I love the phone and all, because I really did...",love phone really need one expect price bill r...
4,4,positive,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,phone great every purpose offer except day buy...


In [ ]:
reviews_df.head()

,rating,sentiment,title,body,review_clean
0,3,negative,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,samsung awhile absolute doo doo read review de...
1,1,negative,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,due software issue nokia sprint phone's text m...
2,5,positive,Love This Phone,"This is a great, reliable phone. I also purcha...",great reliable phone also purchase phone samsu...
3,3,negative,"Love the Phone, BUT...!","I love the phone and all, because I really did...",love phone really need one expect price bill r...
4,4,positive,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,phone great every purpose offer except day buy...


In [ ]:
target_values={'negative':0 ,'positive':1}
target=reviews_df['sentiment']
reviews_df.drop('sentiment',axis=1,inplace=True)
target=target.map(target_values)

In [ ]:
reviews_df=pd.concat([target,reviews_df],axis=1)
reviews_df.head()

,sentiment,rating,title,body,review_clean
0,0,3,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,samsung awhile absolute doo doo read review de...
1,0,1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,due software issue nokia sprint phone's text m...
2,1,5,Love This Phone,"This is a great, reliable phone. I also purcha...",great reliable phone also purchase phone samsu...
3,0,3,"Love the Phone, BUT...!","I love the phone and all, because I really did...",love phone really need one expect price bill r...
4,1,4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,phone great every purpose offer except day buy...


In [ ]:
reviews_df.shape

(67986, 5)

In [ ]:
sentences=reviews_df.review_clean
labels=reviews_df.sentiment

In [ ]:
embedding_dim=100
max_length=16
trunc_type='post'
padding_type='post'
oov_tok='<OOV>'

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index=tokenizer.word_index
vocab_size=len(word_index)
print(vocab_size)
sequences=tokenizer.texts_to_sequences(sentences)
padded=pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

30438


In [ ]:
train_padded,test_padded,train_labels,test_labels=train_test_split(padded,labels,test_size=0.33,random_state=42)

In [ ]:
test_padded, Xval, test_labels, yval = train_test_split(test_padded, test_labels, test_size = 0.5)

In [ ]:
test_padded.shape, Xval.shape, test_labels.shape, yval.shape

((11218, 16), (11218, 16), (11218,), (11218,))

In [ ]:
train_padded = np.array(train_padded)
train_labels = np.array(train_labels)
test_padded = np.array(test_padded)
test_labels = np.array(test_labels)
Xval = np.array(Xval)
yval = np.array(yval)

In [ ]:
train_padded

array([[ 129,  280,  192, ...,    0,    0,    0],
       [  19,  128,    8, ...,    0,    0,    0],
       [ 304,  462,    5, ...,    0,    0,    0],
       ...,
       [   3,   45,  722, ...,    0,    0,    0],
       [   1,  102,   22, ...,    9,  235, 2223],
       [   7, 1055,   64, ..., 2419,    0,    0]], dtype=int32)

In [ ]:
comment_list=list()
lines=reviews_df['review_clean'].values.tolist()

for line in lines:
  tokens=word_tokenize(line)
  comment_list.append(tokens)

In [ ]:
reviews_df.head()

,sentiment,rating,title,body,review_clean
0,0,3,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,samsung awhile absolute doo doo read review de...
1,0,1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,due software issue nokia sprint phone's text m...
2,1,5,Love This Phone,"This is a great, reliable phone. I also purcha...",great reliable phone also purchase phone samsu...
3,0,3,"Love the Phone, BUT...!","I love the phone and all, because I really did...",love phone really need one expect price bill r...
4,1,4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,phone great every purpose offer except day buy...


In [ ]:
print(len(comment_list))

67986


In [ ]:
import gensim
model=gensim.models.Word2Vec(sentences=comment_list,size=100,window=5,workers=4,min_count=1)
words=list(model.wv.vocab)
print(len(words))

36243


In [ ]:
model.wv.most_similar('man')

[('woman', 0.7671648859977722),
 ('boy', 0.710991382598877),
 ('girl', 0.7108829021453857),
 ('toy', 0.7099775075912476),
 ('commando', 0.6989731788635254),
 ('baby', 0.6851880550384521),
 ('mama', 0.6757761240005493),
 ('child', 0.674369215965271),
 ('girlfriend', 0.667829692363739),
 ('manly', 0.6653692722320557)]

In [ ]:
filename='comment_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename,binary=False)

In [ ]:
embeddings_index={};
with open('comment_embedding_word2vec.txt') as f:
  for line in f:
    values=line.split();
    word=values[0];
    coefs=np.asarray(values[1:],dtype='float32');
    embeddings_index[word]=coefs;


embeddings_matrix2=np.zeros((vocab_size+1,embedding_dim));
for word,i in word_index.items():
  embedding_vector=embeddings_index.get(word);
  if embedding_vector is not None:
    embeddings_matrix2[i]=embedding_vector;

In [ ]:
print(embeddings_matrix2)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.11038291e+00 -7.26984620e-01 -1.15612127e-01 ...  3.84337693e-01
  -5.26549101e-01 -1.22144604e+00]
 [ 5.03770292e-01  7.65678644e-01  1.34118581e+00 ...  2.54751182e+00
  -1.74883813e-01  3.26565981e-01]
 ...
 [-1.14263073e-02  1.36066983e-02 -4.74261632e-03 ...  1.44675346e-02
   1.38439424e-02  1.08581213e-02]
 [-7.42369983e-03  1.93995563e-03 -1.74658454e-03 ...  4.89503238e-03
   3.55439819e-03 -5.39794937e-03]
 [ 6.67431392e-03  1.09281717e-02 -9.88502614e-03 ...  3.02806757e-02
  -7.25852966e-04  2.98999762e-03]]


Build of neural netwok

In [ ]:
embeddings_matrix2.shape

(30439, 100)

In [ ]:
embeddings_matrix2[1]

array([-1.11038291, -0.72698462, -0.11561213,  0.80652612, -0.95590973,
       -1.61658955, -0.0869565 , -0.3785699 ,  1.86318672,  0.33643663,
        1.29514742, -0.25121382, -0.26539826, -1.67284715,  0.92462474,
       -0.29598108,  1.15981102,  0.60798174, -0.09781678,  0.00577639,
       -1.84442711, -0.18175291,  1.13723922, -0.61218715,  0.5441016 ,
       -1.33510041,  0.87211752,  0.05114523,  1.0801971 ,  0.58852124,
       -0.54166597, -0.335168  , -2.18167925,  0.05957774,  0.10216238,
        0.35839948,  0.5283699 ,  0.36754337, -0.71267349, -0.32153672,
       -1.41237462, -0.92149836,  1.1156286 ,  0.12336677,  2.35926819,
        1.02784848, -0.28321967,  1.82823992, -1.10600102, -0.02234341,
       -1.46150672,  0.0515752 , -1.24424529, -0.06193374, -2.80277634,
       -1.5634371 ,  0.47423106, -0.62071323, -0.43851879,  0.36174187,
        0.24377923, -0.87786484, -0.23159741,  1.26251304, -0.09520043,
        0.23705319, -0.37321126,  0.16605012, -1.68169665, -0.33

In [ ]:
import numpy as np
df=pd.DataFrame(embeddings_matrix2)
df.head()
data=np.array(embeddings_matrix2)
m,n=data.shape
np.random.shuffle(data)


data_dev=data[0:15000].T
Y_dev=data_dev[0]
X_dev=data_dev[1:n]

data_train=data[15000:m].T
Y_train=data_train[0]
X_train=data_train[1:n]


'data=np.array(embeddings_matrix2)\nm,n=data.shape\nnp.random.shuffle(data)\n\n\ndata_dev=data[0:15000].T\nY_dev=data_dev[0]\nX_dev=data_dev[1:n]\n\ndata_train=data[15000:m].T\nY_train=data_train[0]\nX_train=data_train[1:n]\n'

In [ ]:
Y_train

array([-0.1820278 ,  0.        ,  0.02146918, ..., -0.02017742,
       -0.01230215,  0.00272698])

In [ ]:
X_train[0].shape

(29439,)

In [ ]:
X_train[:,0].shape

(99,)

In [ ]:
import random
import math
def init_params():
  w1=np.random.randn(2,99)-0.5
  b1=np.random.randn(2,1)-0.5
  w2=np.random.randn(2,2)-0.5
  b2=np.random.randn(2,1)-0.5
  return w1,b1,w2,b2

def ReLu(Z):
  return np.maximum(0,Z)

def softmax(Z):

  return np.exp(Z)/np.sum(np.exp(Z))


def forward_prop(w1,b1,w2,b2,X):
  Z1=X.dot(w1)+b1
  A1=ReLu(Z1)
  print(A1)
  print(A1.shape)
  Z2=w2.dot(A1)+b2
  A2=softmax(A1)
  return Z1,A1,Z2,A2

def one_hot(Y):
  n_col=int(np.amax(Y))+1
  binarized=np.zeros((len(Y),int(Y.max())+1))
  binarized[np.arange(len(Y),Y.any())]=1
  binarized=binarized.T
  return binarized

def deriv_ReLU(Z):
  return Z>0

def back_prop(Z1,A1,Z2,A2,w2,X,Y):
  m=Y.size
  one_hot_Y=one_hot(Y)
  dZ2=A2*(one_hot_Y.T)
  dW2=1/m*dZ2.dot(A1.T)
  db2=1/m*np.sum(dZ2,2)
  dZ1=w2.T.dot(dZ2)*deriv_ReLU(Z1)
  dw1=1/m*dZ1.dot(X.T)
  db1=1/m*np.sum(dz1,2)
  return dw1,db1,dw2,db2
  

def update_params(w1,b1,w2,b2,alpha):
  w1=w1-alpha*dw1
  b1=b1-alpha*db1
  w2=w2-alpha*dw2
  b2=b2-alpha*db2
  return w1,b1,w2,b2

def get_predictions(A2):
  return np.argmax(A2,0)

def get_accuracy(predictions,Y):
  print(predictions,Y)
  return np.sum(predictions==Y)/Y.size

def gradient_descent(X,Y,iterations,alpha):
  w1,b1,w2,b2=init_params()
  #for i in range(iterations):
  Z1,A1,Z2,A2=forward_prop(w1,b1,w2,b2,X)
  dw1,db1,dw2,db2=back_prop(Z1,A1,Z2,A2,w2,X,Y)
  w1,b1,w2,b2=update_params(w1,b1,w2,b2,alpha)
    #if i%10==0:
  print("Iterations:",i)
  print("Accuracy",get_accuracy(get_predictions(A2),Y))
  return w1,b1,w2,b2

w1,b1,w2,b2=gradient_descent(X_train,Y_train,10,0.001)